该bkp文件用于记录调研和修改过程，实际使用请参考 json2amass_npz.ipynb

In [42]:
import numpy as np
import pandas as pd
import json

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)

# mapping_table

In [43]:
import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

In [44]:

def json2amass_npz(json_path):

    pos_x = "position_x"
    pos_y = "position_y"
    pos_z = "position_z"
    rot_x = "rotation_x"
    rot_y = "rotation_y"
    rot_z = "rotation_z"

    with open(json_path,'r',encoding = 'utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male' # 性别
        mocap_framerate = 30 # 帧率
        betas = np.zeros(16) # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8)) # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3)) # 全局平移
        poses = np.zeros((frame_length, 156)) # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]


            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:
                
                joint_name = joint_data["joint_name"]

                if (joint_name == root_joint_name):
                            # 把pos赋给全局平移
                    trans[frame_index, :] = [joint_data[pos_x], 
                                            joint_data[pos_y], 
                                            joint_data[pos_z]]
                else:
                    # 尝试获取当前关节序号
                    try:
                        order = mapping_table.loc[joint_name]["order"]
                    except KeyError:
                        continue

                    poses[frame_index, order*3] = joint_data[rot_x]
                    poses[frame_index, order*3+1] = joint_data[rot_y]
                    poses[frame_index, order*3+2] = joint_data[rot_z]

        # 角度值转弧度制
        poses = np.deg2rad(poses)


        np.savez(get_changed_extention_abs_path(json_path,".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)


In [45]:
data_path = './TestData/'
file_name = 'a020_032100.json'


full_path = data_path + file_name

json2amass_npz(full_path)

In [46]:
import os
import glob

# 指定目录路径
dir_path = r'F:\AI Repository\Motion Diffusion\SoulsAnim\ER'

# 获取目录中所有的 JSON 文件
json_files = glob.glob(os.path.join(dir_path, '*.json'))

i = 0

# 输出 JSON 文件名列表
for file_path in json_files:
    # print(os.path.basename(file_path))
    json2amass_npz(file_path)

    i = i + 1
    # print(i)


KeyboardInterrupt: 

In [ ]:
import os
import shutil

# 源目录和目标目录
source_dir = 'F:\AI Repository\Motion Diffusion\SoulsAnim'
dest_dir = 'F:\AI Repository\Motion Diffusion\SoulsAmass'

# 遍历源目录下的所有文件和子目录
for root, dirs, files in os.walk(source_dir):
    # 对于每个文件
    for file in files:
        # 如果文件是以'.npz'结尾的
        if file.endswith('.npz'):
            # 计算文件的相对路径
            rel_path = os.path.relpath(os.path.join(root, file), source_dir)
            # 构造目标路径
            dest_path = os.path.join(dest_dir, rel_path)
            # 如果目标路径不存在，则创建目录
            if not os.path.exists(os.path.dirname(dest_path)):
                os.makedirs(os.path.dirname(dest_path))
            # 复制文件
            shutil.copyfile(os.path.join(root, file), dest_path)


In [ ]:
# 错的！！！！别用！！！！
# 尝试给四肢引入偏移




import json
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)



def json2amass_npz_nb(json_path, mapping_table):

    with open(json_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male'  # 性别
        mocap_framerate = 30  # 帧率
        betas = np.zeros(16)  # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8))  # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3))  # 全局平移
        poses = np.zeros((frame_length, 156))  # 姿势参数

        kinematic_chains = {
            0: [0, 3, 6, 9, 12, 15],
            1: [1, 4, 7, 10],
            2: [2, 5, 8, 11],
            13: [13, 16, 18, 20, 22],
            14: [14, 17, 19, 21, 23],
        }

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]

            pelvis_position = np.zeros(3)

            joint_positions = {}
            joint_rotations = {}

            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:
                joint_name = joint_data["joint_name"]

                # 尝试获取当前关节序号
                try:
                    order = mapping_table.loc[joint_name]["order"]

                    # 获取欧拉角
                    euler_angles = np.array([joint_data["rotation_x"],
                                            joint_data["rotation_y"],
                                            joint_data["rotation_z"]])

                    euler_angles = (euler_angles + 180) % 360 - 180

                    # 储存匹配全局偏移
                    if joint_name == 'm_avg_Pelvis':
                        pelvis_position = np.array([joint_data["position_x"],
                                                    -joint_data["position_z"],
                                                    joint_data["position_y"]])

                    # 将欧拉角转换为轴角表示法
                    # 1. 将欧拉角转换为四元数表示
                    quaternion = Rotation.from_euler('xyz', euler_angles, degrees=True).as_quat()
                    # 2. 将四元数转换为轴角表示
                    axis_angle = Rotation.from_quat(quaternion).as_rotvec()

                    joint_positions[order] = np.array([joint_data["position_x"],
                                                       joint_data["position_y"],
                                                       joint_data["position_z"]])

                    joint_rotations[order] = axis_angle

                except KeyError:
                    continue


            # 根据动力学链条更新poses
            for idx, chain in kinematic_chains.items():
                for order, next_order in zip(chain[:-1], chain[1:]):
                    
                    # 链条0则不进行偏移
                    if idx == 0:
                        poses[frame_index, order * 3:(order + 1) * 3] = joint_rotations[order]

                    # 其它链条
                    else:
                        # 如果是链条起始, 则根据上一节点计算旋转
                        if order in [1, 2, 13, 14]:
                            if order in [1, 2]:
                                position_diff = joint_positions[order] - joint_positions[0]
                            else:
                                position_diff = joint_positions[order] - joint_positions[9]

                            axis_angle = Rotation.from_rotvec(position_diff).as_rotvec()    
                            poses[frame_index, order * 3:(order + 1) * 3] = axis_angle

                        # 将当前order旋转存至下一节点
                        poses[frame_index, next_order * 3:(next_order + 1) * 3] = joint_rotations[order]

            # 计算m_avg_Pelvis和m_avg_root累计的位置变化
            trans[frame_index, :] = pelvis_position * 0.01

            # 将数据保存为AMASS格式（NPZ文件）
            np.savez(get_changed_extention_abs_path(json_path, ".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)



json2amass_npz_nb(r"C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.json", mapping_table)

import shutil

src_file = r'C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.npz'
dst_file = r'C:\Users\Syan\Documents\GitHub\HumanML3D\test_data\test\111111.npz'

shutil.copyfile(src_file, dst_file)




'C:\\Users\\Syan\\Documents\\GitHub\\HumanML3D\\test_data\\test\\111111.npz'

In [ ]:
# 错的！！！
# 尝试通过距离计算轴角

import json
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)

def json2amass_npz_new(json_path, mapping_table):
    with open(json_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male'  # 性别
        mocap_framerate = 30  # 帧率
        betas = np.zeros(16)  # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8))  # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3))  # 全局平移
        poses = np.zeros((frame_length, 156))  # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]

            pelvis_position = np.zeros(3)
            pelvis_rotation = np.zeros(3)
            joint_positions = {}

            # 首先收集关节位置数据
            for joint_data in frame_data["joint_position"]:
                joint_name = joint_data["joint_name"]
                try:
                    order = mapping_table.loc[joint_name]["order"]
                    joint_positions[order] = np.array([joint_data["position_x"],
                                                        -joint_data["position_z"],
                                                        joint_data["position_y"]])

                    if joint_name == 'm_avg_Pelvis':
                        pelvis_position = joint_positions[order]
                        # 获取欧拉角
                        euler_angles = np.array([joint_data["rotation_x"],
                                                joint_data["rotation_y"],
                                                joint_data["rotation_z"]])
                        euler_angles = (euler_angles + 180) % 360 - 180

                        # 将欧拉角转换为轴角表示法
                        # 1. 将欧拉角转换为四元数表示
                        quaternion = Rotation.from_euler('xyz', euler_angles, degrees=True).as_quat()
                        # 2. 将四元数转换为轴角表示
                        pelvis_rotation = Rotation.from_quat(quaternion).as_rotvec()

                except KeyError:
                    continue

            kinematic_chains = [
                [0, 2, 5, 8, 11],
                [0, 1, 4, 7, 10],
                [0, 3, 6, 9, 12, 15],
                [9, 14, 17, 19, 21, 23],
                [9, 13, 16, 18, 20, 22]
            ]

            # 计算旋转
            for chain in kinematic_chains:
                for i in range(len(chain) - 1):
                    order = chain[i]
                    next_order = chain[i + 1]
                    if order in joint_positions and next_order in joint_positions:
                        position_diff = joint_positions[next_order] - joint_positions[order]
                        axis_angle = Rotation.from_rotvec(position_diff).as_rotvec()

                        poses[frame_index, order * 3:(order + 1) * 3] = axis_angle

            poses[frame_index, 0:3] = pelvis_rotation

            # 计算m_avg_Pelvis和m_avg_root累计的位置变化
            trans[frame_index, :] = pelvis_position * 0.01

    # 将数据保存为AMASS格式（NPZ文件）
    np.savez(get_changed_extention_abs_path(json_path, ".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)


json2amass_npz_new(r"C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.json", mapping_table)

import shutil

src_file = r'C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.npz'
dst_file = r'C:\Users\Syan\Documents\GitHub\HumanML3D\test_data\test\111111.npz'

shutil.copyfile(src_file, dst_file)

'C:\\Users\\Syan\\Documents\\GitHub\\HumanML3D\\test_data\\test\\111111.npz'

In [64]:
# 稳定版

import json
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)



def json2amass_npz(json_path, mapping_table):

    with open(json_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male'                         # 性别
        mocap_framerate = 30                    # 帧率
        betas = np.zeros(16)                    # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8))     # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3))     # 全局平移
        poses = np.zeros((frame_length, 156))   # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]

            pelvis_position = np.zeros(3)

            joint_positions = {}
            joint_rotations = {}

            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:

                joint_name = joint_data["joint_name"]
                # 尝试获取当前关节序号
                try:

                    order = mapping_table.loc[joint_name]["order"]
                    joint_positions[order] = np.array([joint_data["position_x"],
                                                    -joint_data["position_z"],
                                                    joint_data["position_y"]])

                    if joint_name == 'm_avg_Pelvis':
                        pelvis_position = joint_positions[order]

                    # 获取欧拉角
                    euler_angles = np.array([joint_data["rotation_x"],
                                            joint_data["rotation_y"],
                                            joint_data["rotation_z"]])

                    euler_angles = (euler_angles + 180) % 360 - 180
                    # 将欧拉角转换为轴角表示法
                    joint_rotations[order] = Rotation.from_euler('xyz', euler_angles, degrees=True).as_rotvec()
                        
                except KeyError:
                    continue
                
                
            for order in joint_rotations.keys():
                poses[frame_index, order * 3:(order + 1) * 3] = joint_rotations[order]

            # 计算m_avg_Pelvis和m_avg_root累计的位置变化
            
            trans[frame_index, :] = pelvis_position * 0.01

    # 将数据保存为AMASS格式（NPZ文件）
    np.savez(get_changed_extention_abs_path(json_path, ".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)


json2amass_npz(r"C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.json", mapping_table)

import shutil

src_file = r'C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.npz'
dst_file = r'C:\Users\Syan\Documents\GitHub\HumanML3D\test_data\test\111111.npz'

shutil.copyfile(src_file, dst_file)




'C:\\Users\\Syan\\Documents\\GitHub\\HumanML3D\\test_data\\test\\111111.npz'

In [91]:
import json
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)



def json2amass_npz_new(json_path, mapping_table):

    with open(json_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male'  # 性别
        mocap_framerate = 30  # 帧率
        betas = np.zeros(16)  # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8))  # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3))  # 全局平移
        poses = np.zeros((frame_length, 156))  # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]

            pelvis_position = np.zeros(3)

            joint_positions = {}
            joint_rotations = {}

            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:
                joint_name = joint_data["joint_name"]



                # 尝试获取当前关节序号
                try:
                    order = mapping_table.loc[joint_name]["order"]

                    joint_positions[order] = np.array([joint_data["position_x"],
                                                       joint_data["position_y"],
                                                       joint_data["position_z"]])

                    if joint_name == 'm_avg_Pelvis':
                        pelvis_position = np.array([joint_data["position_x"],
                                                    -joint_data["position_z"],
                                                    joint_data["position_y"]])

                    # 获取欧拉角
                    euler_angles = np.array([joint_data["rotation_x"],
                                            joint_data["rotation_y"],
                                            joint_data["rotation_z"]])

                    # 大迷惑事件
                    if joint_name == 'm_avg_R_Collar':
                        euler_angles = euler_angles + [0, 0, 0]
                        euler_angles = euler_angles + [0, 180, 0]

                    euler_angles = (euler_angles + 180) % 360 - 180
                    # 将欧拉角转换为轴角表示法
                    joint_rotations[order] = Rotation.from_euler('xyz', euler_angles, degrees=True).as_rotvec()
                        
                except KeyError:
                    continue
                
                
            for order in joint_rotations.keys():

                # 巨大迷惑
                #if order == 14:
                    #position_diff = joint_positions[17] - joint_positions[order]
                    #joint_rotations[order] = Rotation.from_rotvec(position_diff).as_rotvec()

                poses[frame_index, order * 3:(order + 1) * 3] = joint_rotations[order]

            # 计算m_avg_Pelvis和m_avg_root累计的位置变化
            
            trans[frame_index, :] = pelvis_position * 0.01

    # 将数据保存为AMASS格式（NPZ文件）
    np.savez(get_changed_extention_abs_path(json_path, ".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)


json2amass_npz_new(r"C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.json", mapping_table)

import shutil

src_file = r'C:\Users\Syan\Documents\GitHub\Fbx2SMPL\TestData\a020_032100.npz'
dst_file = r'C:\Users\Syan\Documents\GitHub\HumanML3D\test_data\test\111111.npz'

shutil.copyfile(src_file, dst_file)




'C:\\Users\\Syan\\Documents\\GitHub\\HumanML3D\\test_data\\test\\111111.npz'